# DS-SF-27 | Codealong 12 | Decision Trees and Random Forests

In [32]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import math

## Part A - The 2008 Democratic Primaries

(dataset adapted from http://www.stat.ucla.edu/~cocteau/primaries.csv)

In [33]:
df = pd.read_csv(os.path.join('..', 'datasets', '2008-democrat-primaries.csv'))

In [34]:
df.columns

Index([u'fips', u'county_name', u'state_postal', u'region', u'election_date',
       u'racetype', u'tvotes', u'clinton', u'obama', u'edwards', u'margin',
       u'winner', u'POP05_SQMI', u'popUnder30_00', u'pop65up_00',
       u'presVote04', u'kerry04', u'Bush04', u'pres04margin', u'pres04winner',
       u'pop06', u'pop00', u'hisp06', u'white06', u'black06', u'indian06',
       u'asian06', u'hawaii06', u'mixed06', u'pct_less_30k', u'pct_more_100k',
       u'pct_hs_grad', u'pct_labor_force', u'pct_homeowner', u'unempFeb07',
       u'unempFeb08', u'unempChg', u'pctUnins00', u'subForPctHomes',
       u'poverty05', u'median_hhi05', u'Catholic', u'So.Bapt.Conv',
       u'Un.Methodist', u'E.L.C.A.', u'Construction', u'Manufacturing',
       u'FinancialActivities', u'GoodsProducing', u'ServiceProviding'],
      dtype='object')

In [35]:
# TODO
df.winner.unique()

array(['obama', 'clinton', nan], dtype=object)

In [36]:
#TODO
df['c'] =(df.winner == 'obama')

In [37]:
# validate that 20% of blacks voted for Obama
df.black06

0        8559.0
1       16301.0
2       13035.0
3        4705.0
4         872.0
         ...   
2256      347.0
2257       39.0
2258       20.0
2259        8.0
2260        9.0
Name: black06, dtype: float64

### First cut: Is a county more than 20% black?

In [38]:
# TODO
df['pct_black06'] = df.black06 / df.pop06

In [39]:
parent_df = df

In [40]:
left_child_df = parent_df[parent_df.pct_black06 <= .2]

In [41]:
right_child_df = parent_df.drop(left_child_df.index)

In [42]:
(right_child_df.c == 1).sum()

381

In [43]:
(right_child_df.c == 0).sum()

70

#### First cut/right node

In [44]:
# TODO

In [45]:
def obama_vs_clinton(df):
    obama = (df.c == 1).sum()
    clinton = (df.c == 0).sum()
    if obama > clinton:
        print 'Obama wins these counties {} to {}.'.format(obama, clinton)
    elif clinton > obama:
        print 'Clinton wins these counties {} to {}.'.format(clinton, obama)
    else:
        print 'Obama and Clinton tie in these counties {} {}.'.format(obama, clinton)

In [46]:
obama_vs_clinton(right_child_df)

Obama wins these counties 381 to 70.


### Second cut: Is high school graduation rate higher than 78%?

In [47]:
# TODO
parent_df=right_child_df
left_child_df = parent_df[parent_df.pct_hs_grad <= .78]
right_child_df = parent_df.drop(left_child_df.index)

In [48]:
obama_vs_clinton(left_child_df)

Obama wins these counties 300 to 67.


### Third cut: Is high school graduation rate higher than 87%?

In [49]:
# TODO
parent_df = right_child_df
left_child_df = parent_df[parent_df.pct_hs_grad <= .87]
right_child_df = parent_df.drop(left_child_df.index)

In [50]:
obama_vs_clinton(right_child_df)
#the result is wrong...177 to 36 is the correct response

Obama wins these counties 6 to 0.


## Part B - Building the 2008 Democratic Primaries Decision Tree by Hand

In [20]:
class Node:

    @staticmethod
    def root(root_df):
        cs = sorted(set(root_df.c))
        return Node(cs, root_df)

    def decision(self, left_filter):
        # Collect the observations for which the decision split is true and
        # create the corresponding left node

        left_filter = left_filter(self.df)
        left_df = self.df[left_filter]
        self.left = Node(self.cs, left_df)

        # Same thing on the right side but for the observations that don't
        # satisfy the decision split (the "else")

        right_df = self.df.drop(left_df.index)
        self.right = Node(self.cs, right_df)

        # The entropy after the decision split is the weighted average of the
        # children entropy

        self.after = (self.left.samples * self.left.before
                      + self.right.samples * self.right.before) / self.samples

        # The information gain corresponds to the entropy lost between the
        # parent node (this node and the "before") and its child (the "after")

        self.information_gain = self.before - self.after

        return self

    def __init__(self, cs, df):
        self.cs = cs
        self.df = df

        # Counts of the remaining observations in the subspace per classes
        self.counts = [(self.df.c == c).sum() for c in self.cs]

        # Number of observations in the subspace
        self.samples = sum(self.counts)

        # For empty subspaces, probabilties and entropy are set to zero
        if self.samples == 0:
            self.probabilities = [.0 for count in self.counts]
            self.before = .0
        else:
            self.probabilities = [1. * count / self.samples for count in self.counts]
            self.before = - sum(map(lambda p: p * math.log(p, 2),
                                    filter(lambda p : p > .0, self.probabilities)))

    def status(self):
        print 'classes                       =', self.cs
        print 'before:'
        print "\tparent:"
        print "\t\tsamples       =", self.samples
        print "\t\tcounts        =", self.counts
        print "\t\tprobabilities =", self.probabilities
        print "\t\tentropy       =", self.before
        print 'after:'
        print "\tleft child:"
        print "\t\tsamples       =", self.left.samples
        print "\t\tcounts        =", self.left.counts
        print "\t\tprobabilities =", self.left.probabilities
        print "\t\tentropy       =", self.left.before
        print "\tright child:"
        print "\t\tsamples       =", self.right.samples
        print "\t\tcounts        =", self.right.counts
        print "\t\tprobabilities =", self.right.probabilities
        print "\t\tentropy       =", self.right.before
        print
        print 'before entropy                =', self.before
        print 'after entropy                 =', self.after
        print 'information gain              =', self.information_gain

In [21]:
df.c = df.winner

### First cut

In [51]:
# TODO
node = Node.root(df)

#### Candidate #1: Is a county more than 20% black?

In [52]:
# TODO
node.decision(lambda df: df.pct_black06 <= .2).status()
# this is wrong output. Classes should be nan, Clinton, obama instead of false & true

classes                       = [False, True]
before:
	parent:
		samples       = 2261
		counts        = [1230, 1031]
		probabilities = [0.5440070765148165, 0.45599292348518355]
		entropy       = 0.994404850929
after:
	left child:
		samples       = 1810
		counts        = [1160, 650]
		probabilities = [0.6408839779005525, 0.35911602209944754]
		entropy       = 0.941946973925
	right child:
		samples       = 451
		counts        = [70, 381]
		probabilities = [0.15521064301552107, 0.844789356984479]
		entropy       = 0.622727770437

before entropy                = 0.994404850929
after entropy                 = 0.878272555184
information gain              = 0.116132295745


#### Candidate #2: Is high school graduation rate higher than 78%?

In [53]:
# TODO
node.decision(lambda df: df.pct_hs_grad <= .78).status()

classes                       = [False, True]
before:
	parent:
		samples       = 2261
		counts        = [1230, 1031]
		probabilities = [0.5440070765148165, 0.45599292348518355]
		entropy       = 0.994404850929
after:
	left child:
		samples       = 1174
		counts        = [781, 393]
		probabilities = [0.6652470187393527, 0.33475298126064734]
		entropy       = 0.919708944493
	right child:
		samples       = 1087
		counts        = [449, 638]
		probabilities = [0.41306347746090155, 0.5869365225390984]
		entropy       = 0.978081108578

before entropy                = 0.994404850929
after entropy                 = 0.947771988438
information gain              = 0.0466328624907


#### Candidate #3: Is high school graduation rate higher than 87%?

In [54]:
# TODO
node.decision(lambda df: df.pct_hs_grad <= .87).status()

classes                       = [False, True]
before:
	parent:
		samples       = 2261
		counts        = [1230, 1031]
		probabilities = [0.5440070765148165, 0.45599292348518355]
		entropy       = 0.994404850929
after:
	left child:
		samples       = 2042
		counts        = [1194, 848]
		probabilities = [0.584720861900098, 0.4152791380999021]
		entropy       = 0.979189497469
	right child:
		samples       = 219
		counts        = [36, 183]
		probabilities = [0.1643835616438356, 0.8356164383561644]
		entropy       = 0.644694043812

before entropy                = 0.994404850929
after entropy                 = 0.946790335881
information gain              = 0.0476145150476


### Second cut

In [26]:
# TODO


### Third cut

In [27]:
# TODO